# RAG中小块检索返回对应的大块
主要有两种技术：
- 较小的子块引用较大的父块：在检索过程中首先获取较小的块，然后引用父ID，并返回较大的块；
- 句子窗口检索：在检索过程中获取一个句子，并返回句子周围的文本窗口。

## 安装包、设置环境和下载数据集

In [ ]:
! pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision


import os
os.environ["OPENAI_API_KEY"] = "TYPE YOUR API KEY HERE"

!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"


## 导入所需的包

In [ ]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader, Document
from llama_index.response.notebook_utils import display_source_node
from llama_index.retrievers import RecursiveRetriever
from llama_index.core.embeddings import resolve_embed_model
from llama_index.schema import IndexNode
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext, SimpleNodeParser
from llama_index.llms import OpenAI
import json

## 加载文档
使用PDFReader加载PDF文件，并将文档的每一页合并为一个document对象。

In [ ]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("llama2.pdf"))
doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

## 将文档解析为文本块（节点）
将文档拆分为文本块，这些文本块在LlamaIndex中被称为“节点”，将chuck大小定义为1024。默认的节点ID是随机文本字符串，然后可以将节点ID格式化为遵循特定的格式。

In [ ]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)  # 将文档分块，每块1024个字符
base_nodes = node_parser.get_nodes_from_documents(docs)  # 从文档中提取节点
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

## 选择embedding模型和LLM
需要定义两个模型：
- embedding模型用于为每个文本块创建矢量嵌入。在这里使用Hugging Face中的FlagEmbedding模型；
- LLM：用户查询和相关文本块喂给LLM，让其生成具有相关上下文的答案。

可以在ServiceContext中将这两个模型捆绑在一起，并在以后的索引和查询步骤中使用它们。

In [ ]:
embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

## 创建索引、检索器和查询引擎

索引、检索器和查询引擎是基于用户数据或文档进行问答的三个基本组件：

- 索引是一种数据结构，使我们能够从外部文档中快速检索用户查询的相关信息。矢量存储索引获取文本块/节点，然后创建每个节点的文本的矢量嵌入，以便LLM查询。

In [ ]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)

- Retriever用于获取和检索给定用户查询的相关信息。

In [ ]:
base_retriever = base_index.as_retriever(similarity_top_k=2)

- 查询引擎建立在索引和检索器之上，提供了一个通用接口来询问有关数据的问题

In [ ]:
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)
response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

## 高级方法1：较小的子块参照较大的父块

In [ ]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)
all_nodes_dict = {n.node_id: n for n in all_nodes}

### 创建索引、检索器和查询引擎
- 索引：创建所有文本块的矢量嵌入。

In [ ]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)

- Retriever：这里的关键是使用RecursiveRetriever遍历节点关系并基于“引用”获取节点。这个检索器将递归地探索从节点到其他检索器/查询引擎的链接。对于任何检索到的节点，如果其中任何节点是IndexNodes，则它将探索链接的检索器/查询引擎并查询该引擎

In [ ]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

- 创建一个查询引擎作为通用接口来询问有关数据的问题

In [ ]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

## 高级方法2：语句窗口检索

- 创建句子窗口节点解析器

In [ ]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,  # window_size表示在每个句子周围的句子数量
    window_metadata_key="window",  # window_metadata_key表示在节点中存储窗口的键
    original_text_metadata_key="original_text",  # original_text_metadata_key表示在节点中存储原始文本的键
)
sentence_nodes = node_parser.get_nodes_from_documents(docs)
sentence_index = VectorStoreIndex(sentence_nodes, service_context=service_context)

- 创建查询引擎

In [ ]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(window_response)